## Fetch Rewards Exercise


In this notebook I re-structured the receipts.json, brands.json, and users.json into a structured, tabular format. After turning the json files into tabular format, I identified some data quality issues worth investigating and asked a clarifying from the business stakeholder.

In [175]:
import pandas as pd
import datetime
import numpy as np
from sqlalchemy import create_engine

import sqlite3


In [73]:
## Reading the json files

users = pd.read_json('/Users/faadil/Downloads/users.json', lines=True)
brands = pd.read_json('/Users/faadil/Downloads/brands.json', lines=True)
receipts = pd.read_json('/Users/faadil/Downloads/receipts.json', lines=True)

In [ ]:
## User Table

In [74]:
users.dtypes

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object

In [75]:
## Extracting oid values

users['_id'] = users['_id'].apply(lambda x: x['$oid'])

In [76]:
## Extracting Date time values

def cast_to_datetime(data, col):
    ans=[]
    for x in data[col].values:
        if type(x) == float:
            ans.append(np.nan)
        else:
            ans.append(pd.to_datetime(datetime.datetime.fromtimestamp(x['$date'] // 1000)))
    data[col] = ans
    return data


date_columns = ['createdDate', 'lastLogin']

for col in date_columns:
    users = cast_to_datetime(users, col)

In [77]:
users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 10:25:30,2021-01-03 10:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 10:24:04,2021-01-03 10:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 09:21:22,2021-03-05 11:52:23,fetch-staff,NaN,NaN
491,54943462e4b07e684157a532,True,2014-12-19 09:21:22,2021-03-05 11:52:23,fetch-staff,NaN,NaN
492,54943462e4b07e684157a532,True,2014-12-19 09:21:22,2021-03-05 11:52:23,fetch-staff,NaN,NaN
493,54943462e4b07e684157a532,True,2014-12-19 09:21:22,2021-03-05 11:52:23,fetch-staff,NaN,NaN


In [201]:
users.to_csv('/Users/faadil/Desktop/assesement/users.csv', index=False)

In [ ]:
## Receipts table

In [78]:
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'])

date_cols = ['createDate', 'dateScanned', 'finishedDate',
             'modifyDate', 'pointsAwardedDate', 'purchaseDate']


for col in date_cols:
  receipts = cast_to_datetime(receipts, col)


In [79]:
receipts

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:31,2021-01-03 10:25:36,2021-01-03 10:25:31,500.0,2021-01-02 19:00:00,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:43,2021-01-03 10:24:48,2021-01-03 10:24:43,150.0,2021-01-02 10:24:43,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 10:25:37,2021-01-03 10:25:37,NaT,2021-01-03 10:25:42,NaT,5.0,2021-01-02 19:00:00,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:34,2021-01-03 10:25:39,2021-01-03 10:25:34,5.0,2021-01-02 19:00:00,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 10:25:06,2021-01-03 10:25:06,2021-01-03 10:25:11,2021-01-03 10:25:11,2021-01-03 10:25:06,5.0,2021-01-02 10:25:06,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 05:22:27,2021-03-01 05:22:27,NaT,2021-03-01 05:22:28,NaT,25.0,2020-08-16 20:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 10:42:41,2021-03-01 10:42:41,NaT,2021-03-01 10:42:41,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01 09:07:37,2021-03-01 09:07:37,NaT,2021-03-01 09:07:37,NaT,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 08:07:28,2021-03-01 08:07:28,NaT,2021-03-01 08:07:29,NaT,25.0,2020-08-16 20:00:00,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [202]:
receipts.to_csv('/Users/faadil/Desktop/assesement/receipts.csv', index=False)

In [105]:
## Creating item table from rewardsReceiptItemList in receipts data


item = pd.DataFrame()
item['oid'] = ''
id=[]

for i in range(len(receipts['rewardsReceiptItemList'])):
  x = receipts['rewardsReceiptItemList'].iloc[i]
  if type(x) != float:
    for y in x:
      item = pd.concat([item, pd.DataFrame.from_dict(y, orient='index').transpose()])
      item['oid'] = item['oid'].fillna(receipts.iloc[i]['_id'])

In [117]:
item

,oid,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,603cc2bc0a720fde100003e9,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
0,603cc0630a720fde100003e6,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
0,603cc0630a720fde100003e6,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
0,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [119]:
item.dtypes

oid                                   object
barcode                               object
description                           object
finalPrice                            object
itemPrice                             object
needsFetchReview                      object
partnerItemId                         object
preventTargetGapPoints                object
quantityPurchased                     object
userFlaggedBarcode                    object
userFlaggedNewItem                    object
userFlaggedPrice                      object
userFlaggedQuantity                   object
needsFetchReviewReason                object
pointsNotAwardedReason                object
pointsPayerId                         object
rewardsGroup                          object
rewardsProductPartnerId               object
userFlaggedDescription                object
originalMetaBriteBarcode              object
originalMetaBriteDescription          object
brandCode                             object
competitor

In [203]:
item.to_csv('/Users/faadil/Desktop/assesement/item.csv', index=False)

In [ ]:
## Brands Table

In [133]:
brands = pd.read_json('/Users/faadil/Downloads/brands.json', lines=True)

In [134]:
## Extravting oig and ref from cpg column

brands['_id'] = brands['_id'].apply(lambda x: x['$oid'])
cpg = pd.DataFrame()
id=[]
oid=[]
ref=[]

for i in range(len(brands['cpg'])):
  x = brands['cpg'].iloc[i]
  if type(x) != float:
      oid.append(brands['cpg'][i]['$id']['$oid'])
      ref.append(brands['cpg'][i]['$ref'])
      id.append(brands.iloc[i]['_id'])

cpg['id'] = id
cpg['oid'] = oid
cpg['ref'] = ref

brands = brands.merge(cpg, left_on='_id', right_on='id', how='left').drop('id', axis=1)

In [135]:
brands

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,oid,ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,"{'$ref': 'Cogs', '$id': {'$oid': '5f77274dbe37...",test brand @1601644365844,NaN,NaN,5f77274dbe37ce6b592e90bf,Cogs
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '53e10d6368ab...",Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,53e10d6368abd3c7065097cc,Cogs
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,"{'$ref': 'Cogs', '$id': {'$oid': '5332fa12e4b0...",test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,5332fa12e4b03c9a25efd1e7,Cogs
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,"{'$ref': 'Cogs', '$id': {'$oid': '5332f5f6e4b0...",LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,5332f5f6e4b03c9a25efd0b4,Cogs


In [204]:
brands.to_csv('/Users/faadil/Desktop/assesement/brands.csv', index=False)

## Quality Check


In [137]:
## Issue 1 Duplicated values

In [146]:
duplicatedValues_users = users[users.duplicated()]

In [152]:
print(duplicatedValues_users)

                          _id  active         createdDate           lastLogin  \
1    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04 2021-01-03 10:25:37   
2    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04 2021-01-03 10:25:37   
4    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04 2021-01-03 10:25:37   
5    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04 2021-01-03 10:25:37   
8    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 10:24:04 2021-01-03 10:25:37   
..                        ...     ...                 ...                 ...   
490  54943462e4b07e684157a532    True 2014-12-19 09:21:22 2021-03-05 11:52:23   
491  54943462e4b07e684157a532    True 2014-12-19 09:21:22 2021-03-05 11:52:23   
492  54943462e4b07e684157a532    True 2014-12-19 09:21:22 2021-03-05 11:52:23   
493  54943462e4b07e684157a532    True 2014-12-19 09:21:22 2021-03-05 11:52:23   
494  54943462e4b07e684157a532    True 2014-12-19 09:21:22 2021-03-05 11:52:23   

            role signUpSour

len(users)

In [219]:
len(users)

495

In [155]:
## Issue 2 Null values in the Data set

In [158]:
users.isnull().sum().sort_values(ascending=True)

_id              0
active           0
createdDate      0
role             0
signUpSource    48
state           56
lastLogin       62
dtype: int64

In [159]:
item.isnull().sum().sort_values(ascending=True)

oid                                      0
partnerItemId                            0
finalPrice                             174
itemPrice                              174
quantityPurchased                      174
description                            381
discountedItemPrice                   1172
originalReceiptItemText               1181
barcode                               3851
brandCode                             4341
rewardsProductPartnerId               4672
rewardsGroup                          5210
pointsPayerId                         5674
priceAfterCoupon                      5985
pointsEarned                          6014
metabriteCampaignId                   6078
needsFetchReview                      6128
competitiveProduct                    6296
targetPrice                           6563
preventTargetGapPoints                6583
pointsNotAwardedReason                6601
userFlaggedBarcode                    6604
userFlaggedNewItem                    6618
userFlagged

In [160]:
receipts.isnull().sum().sort_values(ascending=True)

_id                          0
createDate                   0
dateScanned                  0
modifyDate                   0
rewardsReceiptStatus         0
userId                       0
totalSpent                 435
rewardsReceiptItemList     440
purchaseDate               448
purchasedItemCount         484
pointsEarned               510
finishedDate               551
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsAwardedDate          582
dtype: int64

In [218]:
brands.isnull().sum().sort_values(ascending=True)

_id               0
barcode           0
cpg               0
name              0
oid               0
ref               0
category        155
brandCode       234
topBrand        612
categoryCode    650
dtype: int64

## Communicate with Stakeholders




Hi all,

Hope this finds you well.

This is Faadil. I've been working on cleaning up the data that I was given relating to users, brands and receipts a few days ago, and I wanted to give you all an update on my progress and get my questions clarified.

There are some issues within the data that I would like to get some feedback on to make sure what I delivered would be of help:

1. Is there a complete list of the items that contain the item-specific information (such as its original price, original description, etc.) available? I would like this data to potentially reduce the amount of space needed to establish this database we need.

2. There are quite a few missing values in the dataset as I examined each of the columns. For example, a lot of items in the items list don't have an associated barCode. This might lead to problems when creating database relationships. Therefore, I wanted to clarify the relationship between itemNumber, brandCode, and barCode, to make sure I designed the database correctly. There are numerious similar missing values in the remaining tables as well.

3. I have also identified that more than half of the user records(out of a total of 495) are reduntant. This might lead to problems with space allocation when creating the database. Would like to discuss a resolution for this.

4. Lastly, as the number of rows adds up, the current approach I'm taking may take up more time and require more resources, and a traditional, relational database might not be the best solution. Is it possible to switch to other non-traditional data storage systems, such as NoSQL, since NoSQL is good at tackling json-formatted files?

Let me know what your thoughts are. I'd be happy to schedule a meeting to clarify these questions if you think a quick meeting would help.

Looking forward to hearing from you.

Best Regards,
Faadil.